## Gauss Elimination
Given equation

$$Ax=b$$

like

$$
\begin{bmatrix}
6 & -2 & 2 & 4\\
12 & -8 & 6 & 10\\
3 & -13 & 9 & 3\\
-6 & 4 & 1 & -18\\
\end{bmatrix}
\begin{bmatrix}
x_1\\
x_2\\
x_3\\
x_4
\end{bmatrix}
=
\begin{bmatrix}
12\\
34\\
27\\
-38
\end{bmatrix}
$$

the Gauss elimination turns it to an equivalent (ie having the same solution) equation

$$Ux=b^\star$$

where

$$
U =
\begin{bmatrix}
u_1_1 & u_1_2 & \dots & u_1_n \\
0 & u_2_2  & \dots & u_2_n \\
\vdots & \vdots   & \ddots &  \vdots & \\
0& 0 & \dots & u_n_n\\
\end{bmatrix}
$$
In the above example it will be
$$
\begin{bmatrix}
6 & -2 & 2 & 4\\
0 & -4 & 2 & 2\\
0 & 0 & 2 & -5\\
0 & 0 & 0 & -3\\
\end{bmatrix}
\begin{bmatrix}
x_1\\
x_2\\
x_3\\
x_4
\end{bmatrix}
=
\begin{bmatrix}
12\\
10\\
-9\\
-3
\end{bmatrix}
$$
which is very easy to solve with a backward substitution: $x_4= 1$, $2x_3-5\cdot1 = -9$ so $x_3=-2$ etc.


In [113]:
import numpy as np
import plotly.express as px


# A = np.array(
#     [
#         [ 6., -2, 2, 4 ],
#         [ 12, -8, 6, 10 ],
#         [ 3, -13, 9, 3 ],
#         [ -6, 4,  1, -18 ]
#     ]
# )
A = np.array(
    [
        [ 2., 3, -6 ],
        [ 1, -6, 8],
        [ 3, -2, 1]
    ]
)

# U = A.copy()
# U[2,1] = 0.5
b = np.array(
    [ 12., 34, 27 ]
)
# b = np.array(
#     [ 12., 34, 27, -38 ]
# )

def minor(A, i, j):
    return np.delete(np.delete(A, i, 0), j, 1)

def gauss(A: np.ndarray, b: np.ndarray):
    """
    Gaussian elimination
    :param A: Matrix with coefficients of a linear equation set
    :param b: Vector of free constants ('right side')
    :return: (L, U, b1) where A = LU and b1 is a transformed vector of the right side
    """
    n, _ = np.shape(A)
    U = A.copy()
    L = np.diagflat([1. for _ in range(n)])
    b1 = b.copy()
    for i in range(n): # main row
        main_element = U[i, i]
        for j in range(i+1, n):  # rows
            multiplier = U[j, i] / main_element
            for k in range(i, n):  # columns
                U[j, k] = U[j, k] - U[i, k] * multiplier
            b1[j] = b1[j] - b1[i] * multiplier
            L[j, i] = multiplier
    return L, U, b1

def scaled_gauss(A: np.ndarray, b: np.ndarray):
    """
    Gaussian elimination
    :param A: Matrix with coefficients of a linear equation set
    :param b: Vector of free constants ('right side')
    :return: (P, L, U, b1) where PA = LU and b1 is a transformed vector of the right side
    """
    n, _ = np.shape(A)
    U = A.copy()
    p = np.arange(n)
    L = np.diagflat([1. for _ in range(n)])
    s = [ np.max(np.abs(row)) for row in A ]
    b1 = b.copy()
    for i in range(n): # main row
        r = [np.abs(A[j]) / s[j] for j in range(i, n)]
        swapi = np.argmax(r)
        temp = p[i]
        p[i] = p[swapi]
        p[swapi] = temp
        main_element = U[i, i]
        for j in range(i+1, n):  # rows
            pj = p[j]
            multiplier = U[pj, i] / main_element
            for k in range(i, n):  # columns
                U[pj, k] = U[pj, k] - U[i, k] * multiplier
            b1[pj] = b1[pj] - b1[i] * multiplier
            L[pj, i] = multiplier
    return L, U, b1

def solve_backward_substitution(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    """
    Solves a linear equation on assumption that A is an upper triangular matrix
    :param A: an upper triangular matrix of coefficients
    :param b: free variables
    :return: Solution vector
    """
    n, _ = np.shape(A)
    x = np.ndarray(shape = n, dtype='float')
    x[n - 1] =  b[n-1]/A[n-1, n-1]
    for i in range(n-2, -1, -1):
        summ = np.sum([A[i,j] * x[j] for j in range(i+1, n)])
        x[i] = 1. / A[i,i] * (b[i] - summ)
    return x

def solve_forward_substitution(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    """
    Solves a linear equation on assumption that A is a lower triangular matrix
    :param A: a lower triangular matrix of coefficients
    :param b: free variables
    :return: Solution vector
    """
    return solve_backward_substitution(A.transpose(), b)

%run ../print_utils.py

printlatex("A = " + latex_matrix(A))

printlatex("Ax = b")

L, U, b1 = scaled_gauss(A, b)
x = solve_backward_substitution(U, b1)
printlatex(latex_matrix(U.transpose()))
printlatex(latex_matrix(U) + r"\cdot " + latex_matrix(np.array([x]).transpose()) + r" =" + latex_matrix(np.array([b1]).transpose()))
printlatex("A=LU=" + latex_matrix(L) + r"\cdot " +  latex_matrix(U))


$$\begin{align}A = \begin{bmatrix}2.0&3.0&-6.0\\1.0&-6.0&8.0\\3.0&-2.0&1.0\\\end{bmatrix}\end{align}$$

$$\begin{align}Ax = b\end{align}$$

/tmp/ipykernel_276378/2949576472.py:95: RuntimeWarning: divide by zero encountered in double_scalars
  x[i] = 1. / A[i,i] * (b[i] - summ)
/tmp/ipykernel_276378/2949576472.py:95: RuntimeWarning: invalid value encountered in double_scalars
  x[i] = 1. / A[i,i] * (b[i] - summ)


$$\begin{align}\begin{bmatrix}0.0&0.0&3.0\\0.0&0.0&-2.0\\0.0&0.0&1.0\\\end{bmatrix}\end{align}$$

$$\begin{align}\begin{bmatrix}0.0&0.0&0.0\\0.0&0.0&0.0\\3.0&-2.0&1.0\\\end{bmatrix}\cdot \begin{bmatrix}nan\\nan\\27.0\\\end{bmatrix} =\begin{bmatrix}0.0\\0.0\\27.0\\\end{bmatrix}\end{align}$$

$$\begin{align}A=LU=\begin{bmatrix}1.0&0.0&0.0\\0.5&1.0&0.0\\0.0&0.0&1.0\\\end{bmatrix}\cdot \begin{bmatrix}0.0&0.0&0.0\\0.0&0.0&0.0\\3.0&-2.0&1.0\\\end{bmatrix}\end{align}$$